# 6강. 인공작사가 만들기

## 6-3. I 다음 am을 쓰면 반 이상은 맞더라(통계)

In [1]:
sentence = "나는 밥을 먹었다"

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"
# <start>는 문장의 시작
# <start>토큰을 받은 순환 신경망은 "나는" 출력
# 출력된 "나는"을 다시 입력, ... 반복을 통해 "먹었다"까지 출력
# 마지막으로 완성을 뜻하는 <end> 토큰을 생성하여 마무리

print("Sourse 문장:", source_sentence)
print("Target 문장:", target_sentence)

Sourse 문장: <start>나는 밥을 먹었다
Target 문장: 나는 밥을 먹었다<end>


## 6-4. 연극대사 생성 인공지능 만들기_데이터 다듬기

In [2]:
import os, re
import numpy as np
import tensorflow as tf

file_path = os.getenv("HOME") + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [3]:
# 필요없는 문장 지우기
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 길이가 0인 문장 건너뛰기
    if sentence[-1] == ':': continue # 문장의 뒤에서 첫번째 문자가 :인 문장 건너뛰기
        
    if idx > 9 : break # 문장 10개만 확인
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


화자가 표기된 문장(0,3,6), 공백인 문장(2,5,9)은 필요가 없으므로 지워주기

In [15]:
# 문장 전처리
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
    sentence = sentence.strip() # 다시 양쪽 공백 지우기
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>추가
    return sentence
    
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


토큰화(Tokenize) : 문장을 일정한 기준으로 쪼개기  
띄어쓰기를 기준으로 쪼개기
1. 문장부호 -> 문장 부호 양쪽에 공백 추가
2. 대소문자 -> 모든 문자들을 소문자로 변환
3. 특수문자 -> 특수문자 모두 제거

In [16]:
# 정제 데이터 구축하기
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [17]:
# 7000단어를 기억할 수 있는 tokenizer 만들기
def tokenize(corpus):
    # 이미 문장을 정제했기 때문에 filters는 필요없음
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>" # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰주기(문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0cf096de50>


- tf.keras.preprocessing.text.Tokenizer 패키지 : 정제된 데이터를 토큰화하고, 단어 사전을 만들어주며, 데이터를 숫자로 변환(벡터화 vectorize) / 숫자로 변환된 데이터 = 텐서(tensor)  
- 텐서플로우의 Tokenizer와 pad_sequences를 사용 -> 토큰화

In [18]:
# 3번째 행, 10번째 열까지만 출력
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


텐서 데이터는 모두 정수로 이루어져 있다. 이 숫자는 tokenizer에 구축된 단어 사전의 인덱스이다.

In [19]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [20]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것. 사전에는 없지만 0은 바로 패딩 문자 <pad>가 될 것이다.  
소스는 2(start)에서 시작해서 3(end)으로 끝난 후 0(pad)로 채워져 있다.

In [21]:
# 데이터셋 객체를 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

텐서플로우는 model.fit()형태로 numpy array 데이터셋을 생성해 model에 제공하는 형태의 학습이 아닌 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법 사용.  
tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성.

#### 데이터셋 생성 과정
- 정규표현식을 이용한 corpus 생성
- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환


## 6-5. 연극대사 생성 인공지능 만들기_인공지능 학습시키기

![RNN](https://user-images.githubusercontent.com/116326867/203905081-1ea71d25-4dcf-45b3-b8b5-38eefa209bf4.png)

1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성  
tf.keras.Model을 Subclassing하는 방식으로 모델 만들기

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져온다.  
embedding_size = 256 # 워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

입력 텐서에 들어 있는 단어 사전의 인덱스 -> Embedding 레이어 : 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔주기 -> 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용

In [23]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법(model에 데이터를 아주 조금 넣어보는 것)
# model의 input shape이 결정되면서 model.build()가 자동으로 호출됨.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어보기
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-1.60995420e-04,  6.51997179e-05, -2.83668574e-04, ...,
         -3.76122625e-05, -1.59228672e-04,  6.81756137e-05],
        [-7.26288126e-05, -1.42863528e-05, -7.00400269e-04, ...,
          2.10009603e-04, -1.41425917e-04,  6.25086523e-06],
        [-1.87466765e-04, -1.18938944e-04, -1.25731435e-03, ...,
          2.80524750e-04, -3.65418731e-04,  4.08592314e-04],
        ...,
        [-1.60256808e-03,  1.28849142e-03,  3.94108193e-03, ...,
         -9.01996042e-04,  7.79986382e-04, -1.44647865e-03],
        [-1.98059413e-03,  1.52819534e-03,  4.73632896e-03, ...,
         -1.17884774e-03,  9.58196237e-04, -1.56709692e-03],
        [-2.36140564e-03,  1.71841262e-03,  5.49159711e-03, ...,
         -1.47687609e-03,  1.13567058e-03, -1.66125677e-03]],

       [[-1.60995420e-04,  6.51997179e-05, -2.83668574e-04, ...,
         -3.76122625e-05, -1.59228672e-04,  6.81756137e-05],
        [-1.85427809e-04, -2.79486907e-04, -1.

shape = (256, 20, 7001)  
- 7001 : Dense 레이어의 출력 차원수(어느 단어의 확률이 가장 높을지 모델링)  
- 256 : 이전 스텝에서 지정한 배치 사이즈  
- 20 : LSTM에서 자신에게 입력된 시퀀스의 길이(데이터를 입력받으면서 알게됨)

In [24]:
# 모델의 구조를 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


시퀀스의 길이를 모르기 때문에 Output Shape 특정 불가

In [25]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수
    from_logits=True, # 기본값은 False. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다.(softmax함수가 적용되지 않았다는걸 의미) 
    reduction='none'  # 기본값은 SUM. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습.(30번 학습을 반복)

Epoch 1/30
93/93 [==============================] - 20s 194ms/step - loss: 3.4655
Epoch 2/30
93/93 [==============================] - 18s 192ms/step - loss: 2.7990
Epoch 3/30
93/93 [==============================] - 17s 184ms/step - loss: 2.6786
Epoch 4/30
93/93 [==============================] - 17s 182ms/step - loss: 2.5754
Epoch 5/30
93/93 [==============================] - 17s 186ms/step - loss: 2.5166
Epoch 6/30
93/93 [==============================] - 18s 190ms/step - loss: 2.4599
Epoch 7/30
93/93 [==============================] - 17s 187ms/step - loss: 2.4031
Epoch 8/30
93/93 [==============================] - 17s 185ms/step - loss: 2.3493
Epoch 9/30
93/93 [==============================] - 17s 186ms/step - loss: 2.2965
Epoch 10/30
93/93 [==============================] - 17s 188ms/step - loss: 2.2443
Epoch 11/30
93/93 [==============================] - 17s 188ms/step - loss: 2.1949
Epoch 12/30
93/93 [==============================] - 17s 187ms/step - loss: 2.1453
Epoch 13/30
9

참고 : [optimizer, loss1](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) / [optimizer, loss2](https://www.tensorflow.org/api_docs/python/tf/keras/losses)  
어떤 optimizier를 써야할지 모른다면 현재 가장 많이 쓰는 Adam을 써보는 것도 방법.  
얼마나 틀리는지(loss)를 알게하는 함수가 손실함수, 손실함수의 최소값을 찾아가는 과정을 optimization, 이를 수행하는 알고리즘을 optimizer(최적화)

## 6-6. 연극대사 생성 인공지능 만들기_평가하기
평가방법 : BLEU, ROUGE, 사람이 평가하기 등  
자연어 처리에서 결과를 평가할 때는 어떤 단어가 포함 되었는지를 바탕으로 측정.

In [31]:
# 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하는 함수만들기
def generate_text(model, tokenizer, init_sentence='<start>', max_len=20): # 시작 문자열 디폴트는 <start>
    test_input = tokenizer.texts_to_sequences([init_sentence]) # init_sentence 텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64) # 텐서로 변환
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장 만들기(루프를 돌면서 init_sentence에 단어를 하나씩 생성)
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 예측된 값 중 가장 높은 확률인 word index 뽑아내기
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # 예측된 word index를 문장 뒤에 붙이기
        if predict_word.numpy()[0] == end_token: break # 모델이 <end>를 예측했거나
        if test_tensor.shape[1] >= max_len: break # max_len에 도달했다면 문장 생성을 마침
            
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

학습 단계에서는 while문이 필요 없다. 소스 문장과 타겟 문장이 있고 소스 문장을 모델에 입력해서 나온 결과를 타겟 문장과 직접 비교하면 되기 때문.  
BUT 테스트 단계에서는 while문 필요. 텍스트 생성 택스크를 위한 테스트 데이터셋을 따로 생성한 적이 없어 소스 문장과 타겟 문장 없기 때문.

In [32]:
# 시작문장으로 he를 넣어 문장생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he is not lolling on a lewd day bed , <end> '